# Libraries

In [1]:
import pandas as pd
import seaborn as sns
import regex as re

import spacy

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import pyLDAvis
import pyLDAvis.gensim_models as gensim_models
import pyLDAvis.gensim_models as gensimvis


import gensim
# from gensim.models import wrappers
# from gensim.models.wrappers import LdaMallet
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD


import nltk
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer


from pprint import pprint

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore", category=FutureWarning)

/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


# Importing -- Fixing columns

In [2]:
full = pd.read_csv('Data/Data-Cleaned/238k-Uncleaned')

In [3]:
# Renaming tweets column, dropping unnamed column, making tweets strings

full['tweets'] = full['0']
full = full.drop(columns = '0')
full.set_index('Unnamed: 0')
# full['tweets'] = full['tweets'].astype('string')
# full.dropna(inplace = True)

,tweets
Unnamed: 0,
1527025892458999814,@testcranker @ITGuy1959 My solar is 100% off g...
1527025892031086593,Boycott #Tesla #TeslaStock #ElonsProblems Vote...
1527025884690993153,@dinodlz @cmclymer Tesla never would have surv...
1527025878974271489,@MarketRebels @elwalvador It would be funny if...
1527025852008976384,@RiceAndrew Right - it's just his nonsense pos...
...,...
1525643350828777473,"@ValueAnalyst1 Yes, beg - borrow and steal to ..."
1525642529567571969,@iamDCinvestor Owh Have to rethink about my th...
1525639115051515909,"@5050isaloss @cardano_whale Overused example, ..."


# Code for Sentiment ---- ---- NOT NEEDED FOR MODELING

In [99]:
# CODE WORKS --- JUST USE IT LATER 

# Adding Sentiment
sia = SentimentIntensityAnalyzer()
comp_dic = {}
for i, tweet in full['tweets'].iteritems():
    comp_dic[i] = sia.polarity_scores(tweet)['compound']

comp_scores = pd.Series(comp_dic, name = 'sentiment')
df_sent = full.merge(comp_scores, left_index = True, right_index = True)
df = df_sent.set_index('Unnamed: 0')

# Preprocessing ---- will need to test different preprocessing later

In [100]:
"""
The goal here is to get to a final model with the full data and an iteration of cleaned preprocessing
Clean this notebook as much as possible
make a copy of the notebook-- save the original and MUST get bi and trigrams to work


"""

'\nThe goal here is to get to a final model with the full data and an iteration of cleaned preprocessing\nClean this notebook as much as possible\nmake a copy of the notebook-- save the original and MUST get bi and trigrams to work\n\n\n'

In [4]:
def Series_Preprocessor(tweet):
    stopWords = stopwords.words('english')
    stopWords.extend(['tesla', 'c,', 'x', 't', 'p', 'amp'])
    ps = PorterStemmer()
    wnl = WordNetLemmatizer()

        # helper function to change nltk's part of speech tagging to a wordnet format.
    def pos_tagger(nltk_tag):
        if nltk_tag.startswith('J'):
            return wordnet.ADJ
        elif nltk_tag.startswith('V'):
            return wordnet.VERB
        elif nltk_tag.startswith('N'):
            return wordnet.NOUN
        elif nltk_tag.startswith('R'):
            return wordnet.ADV
        else:         
            return None 
    
#   Lowercase all 
    tweet = tweet.lower()
    
#   Removing @ handles, links-- strip whitespace breaks and tabs
    tweet = re.sub(r"@\w+|http\S+", "", tweet).strip().replace("\r", "").replace("\n", "").replace("\t", "")
    
#   Tokenize,make list of words, removing punctuation and stopwords
    tweet = [x for x in word_tokenize(tweet) if ((x.isalpha()) & (x not in stopWords)) ]
    
#   Map part of speech tags to words and use words/tags to lemmatize accuratley
    tweet = list(map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(tweet))) 
    tweet = " ".join([wnl.lemmatize(x[0], x[1]) for x in tweet if x[1] is not None])

    return tweet 

In [ ]:
df = full['tweets'].apply(Series_Preprocessor)

## Need to make Bi and Tri Grams ---- Do this below

#### ============================================================================

#### ============================================================================

## Will need to change entire preprocessing step to make bigrams and trigrams -- 
## Therefore the following code will be using the traw imported data

In [ ]:
# remove punctuation
full['preprep'] = full['tweets'].map(lambda x: re.sub('[,\.!?]', '', x))
# Lowercase
full['preprep'] = full['preprep'].map(lambda x: x.lower())
# #   Removing @ handles, links-- strip whitespace breaks and tabs
full['preprep'] = full['preprep'].map(lambda x: re.sub(r"@\w+|http\S+", "", x).strip().replace("\r", "").replace("\n", "").replace("\t", ""))

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
data = full.preprep.values.tolist()
data_words = list(sent_to_words(data))

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=3, threshold=75) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=60)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
stop_words = stopwords.words('english')
stop_words.extend(['tesla', 'c,', 'x', 't', 'p', 'amp', 'car', 'go'])
wnl = WordNetLemmatizer()

In [ ]:
# Functions to for preprocessing for bigrams and trigrams texts, all tested and edited all should be good 

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words]for doc in texts]

def make_bigrams(texts):
    return bigram_mod[texts]

def make_trigrams(texts):
    return trigram_mod[bigram_mod[texts]]

def pos_tagger(nltk_tag):
    if nltk_tag.startswith('J'):
        return wordnet.ADJ
    elif nltk_tag.startswith('V'):
        return wordnet.VERB
    elif nltk_tag.startswith('N'):
        return wordnet.NOUN
    elif nltk_tag.startswith('R'):
        return wordnet.ADV
    else:         
        return None 
    
def lemmatize_texts(tweet):
     for i in tweet:
        word = (map(lambda x: (x[0], pos_tagger(x[1])), pos_tag(tweet))) 
        word_lem = " ".join([wnl.lemmatize(x[0], x[1]) for x in word if x[1] is not None])
        return word_lem

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# lemmatize
data_lemmatized = [lemmatize_texts(tweet) for tweet in data_words_bigrams]

In [ ]:
#proved that Nones are introduced during the lemmatization step -- there are no nones before this step
# This will need to be fixed later
lem_noNone = [x for x in data_lemmatized if x is not None]

## Find missing values after making min tweet length 4 words maybe =============================

In [ ]:
# testing enumerate with above lemmatization function
tester = [(i,x) for (i,x) in enumerate(data_lemmatized) if x is not None]

def find_missing(lst):
    return [x for x in range(lst[0], lst[-1]+1) 
                               if x not in lst] 

lst = [tup[0] for tup in tester]
ind = (find_missing(lst))

# ind


In [380]:
full['preprep'][1711]

'for what'

In [381]:
len(ind)

659

In [382]:
print(len(data_lemmatized), len(lem_noNone), len(tester))

237659 237000 237000


In [383]:
lem_noNone = [i.split() for i in lem_noNone]

In [ ]:
id2word = corpora.Dictionary(lem_noNone)
print(len(id2word)) 

In [ ]:
# This can be Tuned --- and probably chould be

id2word.filter_extremes(no_below=6, no_above=.90)
print(len(id2word))

In [ ]:
# Another potential preprocessing step 
# Could remove tweets that are fewer than n number of words 

counter = 0
for i in lem_noNone:
    if len(i) <= 3:
        counter += 1
counter

In [ ]:
corpus = [id2word.doc2bow(d) for d in lem_noNone]

In [ ]:
# Instantiating a Base LDA model
base_model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]
topics = [' '.join(t[0:10]) for t in words]

for id, t in enumerate(topics): 
    print(f"------ Topic {id + 1} ------")
    print(t, end="\n\n")

In [389]:
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=lem_noNone, 
                                   dictionary=id2word, coherence='c_v')

coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base, '\n\nPerplexity: ', base_perplexity)


Coherence Score:  0.28988486308685213 

Perplexity:  -7.690942110728512


In [390]:
pyLDAvis.enable_notebook()
gensimvis.prepare(base_model, corpus, id2word)

/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.058025  0.028286       1        1  28.791051
4     -0.063249 -0.128172       2        1  23.596943
3      0.038124 -0.065979       3        1  17.325416
0      0.103780  0.064201       4        1  16.335805
2     -0.136680  0.101664       5        1  13.950785, topic_info=         Term          Freq         Total Category  logprob  loglift
151     stock  25612.000000  25612.000000  Default  30.0000  30.0000
162     model  12276.000000  12276.000000  Default  29.0000  29.0000
128      musk  19576.000000  19576.000000  Default  28.0000  28.0000
1577     fire   8491.000000   8491.000000  Default  27.0000  27.0000
117       car  41816.000000  41816.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
169      need   1509.073659   8656.427470   Topic5  -5.4165   0.2228
151     stock   1728.651386  25612.024388   Topic5  -5.2807  -0.7261
30       sell   1504.819466   9576.793168   Topic5  -5.4193   0.1190
90       want   1508.027155  10253.883317   Topic5  -5.4172   0.0528
154   twitter   1388.017630   8961.175686   Topic5  -5.5001   0.1046

[459 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
289        1  0.217018           also
289        2  0.085097           also
289        3  0.213730           also
289        4  0.310927           also
289        5  0.173351           also
...      ...       ...            ...
355        2  0.197429           year
355        3  0.109096           year
355        4  0.453982           year
355        5  0.057012           year
15694      1  0.989858  youtube_video

[1200 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 5, 4, 1, 3])

#### ============================================================================

# Creating Baseline Below

### Create Gensim Dictionary object
#### Is a value count of all unique tokens

In [104]:
df['tokens'] = [i.split() for i in df]

In [105]:
# Dictionary is a gensim tool
id2word = Dictionary(df['tokens'])
print(len(id2word)) 

44577


###  Filtering Extremes using Gensim 

    **CAN BE CHANGED/ IMPROVED**
    



In [106]:
# id2word.filter_extremes(no_below=4, no_above=.95) #Original -- 12656
id2word.filter_extremes(no_below=6, no_above=.90)
print(len(id2word))

14944


In [107]:
corpus = [id2word.doc2bow(d) for d in df['tokens']]

## LdaMulticore -------------------------

In [108]:
# Instantiating a Base LDA model
base_model = LdaMulticore(corpus=corpus, num_topics=5, id2word=id2word, workers=12, passes=5)
words = [re.findall(r'"([^"]*)"',t[1]) for t in base_model.print_topics()]
topics = [' '.join(t[0:10]) for t in words]

for id, t in enumerate(topics): 
    print(f"------ Topic {id + 1} ------")
    print(t, end="\n\n")

In [111]:
# Getting the topics
for id, t in enumerate(topics): 
    print(f"------ Topic {id + 1} ------")
    print(t, end="\n\n")

------ Topic 1 ------
buy musk elon stock get think go twitter never say

------ Topic 2 ------
car fire electric year make battery drive see ev company

------ Topic 3 ------
model get car first state spot time teslaradar work go

------ Topic 4 ------
stock car company price make market go sell musk electric

------ Topic 5 ------
car door electric people make get crash driver open autopilot



In [112]:
# a measure of how good the model is. lower the better
base_perplexity = base_model.log_perplexity(corpus)

# Compute Coherence Score
coherence_model = CoherenceModel(model=base_model, texts=df['tokens'], 
                                   dictionary=id2word, coherence='c_v')

coherence_lda_model_base = coherence_model.get_coherence()
print('\nCoherence Score: ', coherence_lda_model_base, '\n\nPerplexity: ', base_perplexity)


Coherence Score:  0.292141780223799 
 
 
 
Perplexity:  -7.595869898279568


In [113]:
"""



BASE PREPROCESSIG ON BASE LDA MODEL

without tesla and single letters 
Coherence Score:  0.292141780223799 
Perplexity:  -7.595869898279568


With 'Tesla' and single letters
Coherence Score:  0.35428295570283674 
Perplexity:  -7.273198516919201"""

"\nWith 'Tesla' and single letters\n\n\n\nCoherence Score:  0.35428295570283674 \nPerplexity:  -7.273198516919201"

In [115]:
#Creating Topic Distance Visualization 
pyLDAvis.enable_notebook()
gensimvis.prepare(base_model, corpus, id2word)

/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)
/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/Users/austinboring/opt/anaconda3/envs/broke-the-other/lib/python3.10/site-packages/past/builtins/mi

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.109275  0.030929       1        1  25.297912
1     -0.027695 -0.047618       2        1  23.130874
3      0.106602 -0.050555       3        1  22.726119
4     -0.121650 -0.093700       4        1  16.066342
2     -0.066532  0.160945       5        1  12.778754, topic_info=          Term          Freq         Total Category  logprob  loglift
79         car  63006.000000  63006.000000  Default  30.0000  30.0000
151      stock  25303.000000  25303.000000  Default  29.0000  29.0000
163      model   9919.000000   9919.000000  Default  28.0000  28.0000
1443      door   6628.000000   6628.000000  Default  27.0000  27.0000
144        buy  26310.000000  26310.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
116   electric   1985.010769  20059.994543   Topic5  -5.1251  -0.2557
144        buy   2085.594995  26310.688416   Topic5  -5.0757  -0.4775
119        gas   1222.810207   4332.774557   Topic5  -5.6096   0.7923
19        even   1334.878294   8664.170512   Topic5  -5.5219   0.1870
77         use   1278.828743   9395.651238   Topic5  -5.5648   0.0631

[466 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1467      1  0.984208  accenture
1605      1  0.027602   accident
1605      2  0.050751   accident
1605      3  0.005342   accident
1605      4  0.882358   accident
...     ...       ...        ...
352       1  0.146197       year
352       2  0.588549       year
352       3  0.182477       year
352       4  0.071665       year
352       5  0.011108       year

[1370 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 4, 5, 3])

# Grid Search is below ==================================

# ====================================================================

#### Attempting a GridSearch on the LDA model

In [92]:
df2 = df.drop('tokens')

In [94]:
df2

0         solar grid run computer radio fridge entire ya...
1                boycott teslastock elonsproblems vote blue
2         never survive carbon offset implement democrat...
3                            funny fanbase take profit sell
4         right nonsense position find sort hard believe...
                                ...                        
237654    yes beg borrow steal invest world valuable com...
237655    owh rethink thesis always think regulation bui...
237656    overused example elon musk alienate almost yea...
237657    found spacex successful also arguably innovati...
237658               want ev table musk show world year old
Name: tweets, Length: 237659, dtype: object

In [91]:
vectorizer = CountVectorizer()
data_vectorized = vectorizer.fit_transform(df.drop('tokens'))

In [ ]:
# CV Param
search_params = {'n_components': [5, 10, 15, 20, 25, 30], 'learning_decay': [.5, .7, .9], 'batch_size':[128, 256]}

# Init Model
lda = LatentDirichletAllocation()

# Init Grid Search Class
model = GridSearchCV(lda, cv = 5, param_grid=search_params, n_jobs = -1)

# Grid Search
model.fit(data_vectorized)

In [ ]:
# Best Model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(data_vectorized))

In [ ]:
best_lda_model

In [397]:
id2word.keys()

18507

In [ ]:
GridSearchCV(cv=None, error_score='raise',
             estimator=LatentDirichletAllocation(batch_size=128, 
                                                 doc_topic_prior=None,
                                                 evaluate_every=-1, 
                                                 learning_decay=0.7, 
                                                 learning_method=None,
                                                 learning_offset=10.0, 
                                                 max_doc_update_iter=100, 
                                                 max_iter=10,
                                                 mean_change_tol=0.001, 
                                                 n_components=10, 
                                                 n_jobs=1,
                                                 perp_tol=0.1, 
                                                 random_state=None,
                                                 topic_word_prior=None, 
                                                 total_samples=1000000.0, 
                                                 verbose=0),
             n_jobs=-2,
             param_grid={'n_topics': [10, 15, 20, 30], 
                         'learning_decay': [0.5, 0.7, 0.9]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
             scoring=None, verbose=0)